# Imports

In [ ]:
# Standard library imports
import sys
import os

# Add path
sys.path.append(os.path.join('C:\\','Users','schuen02','Documents', 'Code', 'Python', 'BMCSim'))

# Third party imports
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Local application imports
from BMCtool import BMCTool

# Settings

In [ ]:
%matplotlib qt

# simulation settings
n_timesteps = 1  # number of timesteps for each RF pulse
track = False  # track the magnetization trajectory during pulses/delays ?!
par_calc = True  # calculate all offsets in parallel instead of subsequently

# experimental settings
B0 = 7  # B0 in T
B1 = 1.25e-6*B0  # B1 in T (for WASABI use 1.25e-6*B0)
n_p = 1  # number of saturation pulses
tp = 0.015/B0  # pulse duration in s (for WASABI use 0.015/B0)
td = 0  # delay between pulses in s
shape = 'CW'
trec = 2  # recovery/delay time between offsets in s
offset = 2  # max offset in ppm
n_offsets = 41  # number of offsets

td2 = 0.5

# sample settings
T1 = 2            #T1 in s
T2 = 0.05            #T2 in s

# CEST settings
n_pools = 1

#                bulk       1st       2nd       3rd       4th       5th       6th
T1n = np.array([   T1,       T1,       T1,       T1,       T1,       T1,       T1])
T2n = np.array([   T2,     0.15,     9e-6,     0.15,    0.015,     9e-6,    0.015])
fn  = np.array([    1,    0.002,     0.14,    0.005,    0.005,     0.14,     0.01])
dwn = np.array([ -0.3,      1.3,     -2.6,     -3.3,     -3.7,     -2.6,     -1.0])
kn  = np.array([    0,     2000,       40,     2000,     2000,       40,      500])

# calculate offsets
offsets = np.linspace(-offset, offset, n_offsets)

# Simulation and Noise

In [ ]:
# create Sim object
Sim = BMCTool(b0=B0,
              n_pools=n_pools,
              t1_values=T1n,
              t2_values=T2n,
              poolsizes=fn,
              resonances=dwn,
              exchange_rates=kn,
              track=track)

if par_calc:
    # set initial magnetization
    Sim.set_M(offsets.shape[0], np.zeros([offsets.shape[0], 3, 1]))
 
    # simulate recovery
    Sim.solve(offsets, 0, trec, n_timesteps, shape=shape)

    # simulate saturation pulse train
    for n in range(n_p):
        # delay between pulses (not for the first pulse)
        if n != 0 and td != 0:
            Sim.solve(offsets, b1=0, pulse_dur=td, steps=int(td/tp*n_timesteps))
        # saturation
        Sim.solve(offsets, b1=B1, pulse_dur=tp, steps=int(n_timesteps), shape=shape)
        
    M = np.abs(Sim.M_)
else:
    # create array for magnetization values
    M = np.zeros([len(offsets), 3*n_pools, 1])
    
    # set initial magnetization
    Sim.set_M(1)

    for i in range(offsets.shape[0]):
        # simulate recovery
        if trec > 0:
            Sim.solve(offsets[i], 0, trec, n_timesteps, shape='PAUSE')

        # simulate saturation pulse train
        for n in range(n_p):
            if n != 0 and td != 0:
                Sim.solve(offsets[i], b1=0, pulse_dur=td, steps=int(td/tp*n_timesteps))
            Sim.solve(offsets[i], b1=B1, pulse_dur=tp, steps=int(n_timesteps), shape=shape)
            
        # simulate delay
        Sim.solve(offsets[i], b1=0, pulse_dur=td2, steps=int(n_timesteps), shape=shape)
        
        # write magnetization in array
        M[i,] = np.abs(Sim.M_)

# add noise
sd = 0.01  # standard deviation of noise
M[:,2,0] += np.random.normal(0, sd, offsets.shape[0])

# Plot spektrum (Signal vs. offsets)
## >>> DON'T CLOSE THE FIGURE <<<

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))

ax.plot(offsets, M[:,2,0], marker='.', linestyle='None', linewidth=2, color='black')
ax.set_xlabel('frequency offset [ppm]', fontsize=20)
ax.set_ylabel('normalized signal', fontsize=20)
ax.set_ylim([-0.1,1])
ax.invert_xaxis()
ax.grid()

# Fitting with "old" function
### dB0 is given in ppm here

In [ ]:
from lmfit import Model
from lmfit import models
from lmfit import Parameters

def wasabi(x, dB0, B1, c, d, freq, tp):
    """WASABI function for simultaneous determination of B1 and B0. For more information read:
    Schuenke et al. Magnetic Resonance in Medicine, 77(2), 571–580. https://doi.org/10.1002/mrm.26133"""
    gamma = 42.577
    return np.abs(c - d * np.square(np.sin(np.arctan((B1 / (freq / gamma)) / (x - dB0)))) *
                  np.square(np.sin(np.sqrt(np.square(B1 / (freq / gamma)) +
                                           np.square(x - dB0)) * freq * 2 * np.pi * tp / 2)))

fit_kws = {'ftol': 1.49012e-05}

model = Model(wasabi)
params = model.make_params()
params['freq'].set(42.577 * B0, vary=False)
params['tp'].set(tp, vary=False)
params['dB0'].set(0)
params['B1'].set(B1*1E6*0.9, min=0)
params['c'].set(1)
params['d'].set(1.5)

result1 = model.fit(M[:,2,0], params, x=offsets, max_nfev=100, fit_kws=fit_kws)
# print results
print(result1.fit_report())
# add plot to figure
ax.plot(offsets, result1.best_fit, color='blue', linestyle=':')

# Fitting with "advanced" function
### dB0 is given in rad here

In [ ]:
from lmfit import Model
from lmfit import models
from lmfit import Parameters

def wasabiti(x, dB0, B1, R1, R2, tp, trec, B0):
    gamma_ = 42.577 * 2 * np.pi
    w1 = gamma_ * B1
    dx = x-dB0
    R1p = ((R1*dx**2)/(dx**2 + w1**2) + R2*w1**2/(dx**2 + w1**2))
    R2p = ((2*R2+R1)/2 - R1p/2)  # from https://doi.org/10.1016/j.mri.2013.07.004
    Mzi = 1-np.exp(-R1*trec)
    
    ans = np.abs(Mzi * (dx**2*np.exp(-R1p*tp)/(dx**2+w1**2) + (w1**2*np.cos(tp*np.sqrt(dx**2+w1**2)) * np.exp(-R2p*tp))/(dx**2+w1**2)) +
                 R1 * dx**2 / (R1p*(dx**2+w1**2)) * (1-np.exp(-R1p*tp)) )
    
    return ans

fit_kws = {'maxfev': 100, 'ftol': 1.49012e-05}

model = Model(wasabiti)
params = model.make_params()
params['tp'].set(tp, vary=False)
params['trec'].set(trec, vary=False)
params['B0'].set(B0, vary=False)
params['dB0'].set(0)
params['B1'].set(B1*1E6*0.9, min=0)
params['R1'].set(1, min=0)
params['R2'].set(10, min=0, vary=True)

result2 = model.fit(M[:,2,0], params, x=(offsets * 42.577 * 2 * np.pi * B0), fit_kws=fit_kws)
print(result2.fit_report())
print(f"\n dB0 in ppm = {result2.params.valuesdict()['dB0']/(42.577 * 2 * np.pi * B0)}")
ax.plot(offsets, result2.best_fit, color='r', linestyle='--')